In [712]:
import numpy as np
import pandas as pd
from datetime import datetime

# Import plotly and cufflinks offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

- Import misc.abc_bids report and explore it!
- In this query result I wanted to know all bidders and their bids in each auctio ID on specific auction dates

In [713]:
df = pd.read_csv('report.csv')
df

,auction_date_transformed,auction_id,bstock_category_name,bid_date,bid_id,bidder_id,bid_amount_transformed
0,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:49.000000 UTC,16819417,1,10
1,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:50.000000 UTC,16819418,1,100
2,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:51.000000 UTC,16819419,2,101
3,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:52.000000 UTC,16819420,1,200
4,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:53.000000 UTC,16819421,2,201
...,...,...,...,...,...,...,...
822,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-13 23:58:16.000000 UTC,18712359,5fbc4f53c8ce886dac7f4ba8,500
823,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-14 02:55:27.000000 UTC,18713339,59769fe3e3d4600d362c31c0,708
824,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-14 10:12:08.000000 UTC,18714207,5fbc4f53c8ce886dac7f4ba8,750
825,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-15 16:13:16.000000 UTC,18726173,601e76d8c8ce886da835b151,850


- Clean and convert to appropiate data types

In [714]:
# Clean
df['bid_date'] = df['bid_date'].str.replace('.000000 UTC', '')

# Convert to datetime
df['bid_date'] = pd.to_datetime(df['bid_date'], format="%Y-%m-%d %H:%M:%S")

# Convert to float
df['bid_amount_transformed'] = df['bid_amount_transformed'].astype('float')

# Change columns names
df = df.rename(columns={'auction_date_transformed': 'auction_date',
                        'bid_amount_transformed': 'bid_amount'})

df

/var/folders/zb/60dzf9rs2g5dzbn8q93wz0hw0000gn/T/ipykernel_10892/2427272880.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.



,auction_date,auction_id,bstock_category_name,bid_date,bid_id,bidder_id,bid_amount
0,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:49,16819417,1,10.0
1,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:50,16819418,1,100.0
2,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:51,16819419,2,101.0
3,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:52,16819420,1,200.0
4,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:53,16819421,2,201.0
...,...,...,...,...,...,...,...
822,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-13 23:58:16,18712359,5fbc4f53c8ce886dac7f4ba8,500.0
823,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-14 02:55:27,18713339,59769fe3e3d4600d362c31c0,708.0
824,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-14 10:12:08,18714207,5fbc4f53c8ce886dac7f4ba8,750.0
825,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-15 16:13:16,18726173,601e76d8c8ce886da835b151,850.0


- It is quite difficult to explore the bids amounts distribution on each auction on SQL
- Affortunately, I can explore that at a glance with Python scripts, so first, I have to build a pivot table!

In [715]:
# df_stats_daily
df_stats_daily = df.groupby(['auction_date'])
df_stats_daily = df_stats_daily.agg({'bid_amount': ['count', 'mean', 'std', 'min', 'max']})
df_stats_daily.columns = ['bid_count', 'bid_amount_mean', 'bid_amount_std', 'bid_amount_min', 'bid_amount_max']
df_stats_daily = df_stats_daily.reset_index()
df_stats_daily = df_stats_daily.fillna(0)
df_stats_daily = df_stats_daily.round(2)
df_stats_daily

,auction_date,bid_count,bid_amount_mean,bid_amount_std,bid_amount_min,bid_amount_max
0,2022-09-10,12,492.83,393.04,10.0,1100.0
1,2022-09-13,119,1172.96,912.27,100.0,3000.0
2,2022-09-16,22,833.00,559.96,100.0,2000.0
3,2022-09-23,56,1179.43,685.99,111.0,2800.0
4,2022-09-30,42,559.98,304.75,111.0,1444.0
5,2022-10-07,74,1451.31,745.80,125.0,2888.0
6,2022-11-01,140,2382.99,1418.83,125.0,6222.0
7,2022-11-04,28,2719.86,1327.16,125.0,4000.0
8,2022-11-15,60,1023.37,551.84,111.0,2057.0
9,2022-11-18,47,1045.43,551.87,150.0,2200.0


- I built ,for fun, an inteactive calendar with Python aboout the acount dates and all the bids registered on those dates.
- It is really interesting that most of the auctions were held on Tuesdays and Fridays!

In [716]:
# Daily Calendar of auction dates
df_stats_daily['auction_date'] = pd.to_datetime(df_stats_daily['auction_date'])

new_date_range = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
df_calplot = df_stats_daily.set_index('auction_date')
df_calplot = df_calplot.reindex(new_date_range, fill_value=0)
df_calplot.reset_index(inplace=True)

# calendar
from plotly_calplot import calplot

calplot(df_calplot, x='index', y='bid_count', title='Total bids per acuction date', dark_theme=1, month_lines_width=3, years_title=1)

- Now, I want to know how many bids per auctions were held on these dates!

In [717]:
# Total auctions per auctions date
df_stats_daily_auctions = df.groupby(['auction_date', 'auction_id'])
df_stats_daily_auctions = df_stats_daily_auctions.agg({'bid_amount': ['count', 'mean', 'std', 'min', 'max']})
df_stats_daily_auctions.columns = ['bid_count', 'bid_amount_mean', 'bid_amount_std', 'bid_amount_min', 'bid_amount_max']
df_stats_daily_auctions = df_stats_daily_auctions.reset_index()
df_stats_daily_auctions = df_stats_daily_auctions.fillna(0)
df_stats_daily_auctions = df_stats_daily_auctions.round(2)
df_stats_daily_auctions

,auction_date,auction_id,bid_count,bid_amount_mean,bid_amount_std,bid_amount_min,bid_amount_max
0,2022-09-10,1500000,12,492.83,393.04,10.0,1100.0
1,2022-09-13,1640483,40,1749.10,995.49,100.0,3000.0
2,2022-09-13,1640484,4,153.00,43.73,100.0,207.0
3,2022-09-13,1640485,24,1325.33,833.02,100.0,2521.0
4,2022-09-13,1640486,27,943.11,659.72,100.0,2667.0
5,2022-09-13,1640487,8,397.12,202.39,100.0,622.0
6,2022-09-13,1640488,16,534.81,286.72,101.0,922.0
7,2022-09-16,1646151,11,977.82,675.35,200.0,2000.0
8,2022-09-16,1646152,11,688.18,395.27,100.0,1222.0
9,2022-09-23,1651870,16,875.69,494.09,111.0,1500.0


- September 13th auction was the one that registered the highest auctions (6), and September 10 the lowest auctions (1)!

In [718]:
# Total auctions per auction date plot
df_stats_daily_auctions = df_stats_daily_auctions.pivot(columns='auction_date', values='auction_id').T.count(axis=1)
df_stats_daily_auctions.iplot(kind='bar', yTitle='Count', title='Total auctions per auction date')

- With that in mind, it's intriguing to know which categories were the most requested, don't you think?

In [719]:
# Total bids per category and auction date
df_stats_daily_categories = df.groupby(['auction_date', 'auction_id', 'bstock_category_name'])
df_stats_daily_categories = df_stats_daily_categories.agg({'bid_amount': ['count', 'mean', 'std', 'min', 'max']})
df_stats_daily_categories.columns = ['bid_count', 'bid_amount_mean', 'bid_amount_std', 'bid_amount_min', 'bid_amount_max']
df_stats_daily_categories = df_stats_daily_categories.reset_index()
df_stats_daily_categories = df_stats_daily_categories.fillna(0)
df_stats_daily_categories = df_stats_daily_categories.round(2)
df_stats_daily_categories

,auction_date,auction_id,bstock_category_name,bid_count,bid_amount_mean,bid_amount_std,bid_amount_min,bid_amount_max
0,2022-09-10,1500000,"Apparel, Footwear & Accessories",12,492.83,393.04,10.0,1100.0
1,2022-09-13,1640483,"Apparel, Footwear & Accessories",40,1749.10,995.49,100.0,3000.0
2,2022-09-13,1640484,"Apparel, Footwear & Accessories",4,153.00,43.73,100.0,207.0
3,2022-09-13,1640485,Furniture,24,1325.33,833.02,100.0,2521.0
4,2022-09-13,1640486,Consumer Electronics,27,943.11,659.72,100.0,2667.0
5,2022-09-13,1640487,Consumer Electronics,8,397.12,202.39,100.0,622.0
6,2022-09-13,1640488,Consumer Electronics,16,534.81,286.72,101.0,922.0
7,2022-09-16,1646151,Consumer Electronics,11,977.82,675.35,200.0,2000.0
8,2022-09-16,1646152,"Apparel, Footwear & Accessories",11,688.18,395.27,100.0,1222.0
9,2022-09-23,1651870,"Apparel, Footwear & Accessories",16,875.69,494.09,111.0,1500.0


- On September 13, it had the most auctions because all three categories were sold!
- The same logic applies to the auction on September 10, it is the event with the lowest auctions (1) and only one category ('Apparel, Footwear & Accesories) was sold with a maximum of 12 bids.


# Hold a second!
- I found that on November 1st was the date with the most bids registered ever, even though there were only 2 auctions of 'Appareal, Footwear & Accesories' category!

In [720]:
# Total bids per category and auction date plot
import plotly.express as px
fig = px.bar(df_stats_daily_categories, x='auction_date',
             y='bid_count',
             color='bstock_category_name',
             title='Total bids per category and auction date (each stacked block represent an auction id)',
             text_auto=True)
fig.show()

- So, how many bids does each category have?
- It seems that 'Apparel, Footwear & Accesories have a bunch of bids!

In [721]:
# Total bids per category plot
category_total_bids = df_stats_daily_categories.pivot(columns='bstock_category_name', values='bid_count').T.sum(axis=1).sort_values(ascending=False)
category_total_bids.iplot(kind='bar', yTitle='bid_count', title='Total bids per Category')

- Apparel, Footwear & Accesories category has a total of 741 bids
- Consumer Electrics category has a total of 62 bids
- Furniture category has a total of 24 bids

- Is it possible that the sale of clothing, shoes and accessories generates a lot of profitability?
- That is a question to which I will return later. Let's continue exploring the bids amount distribution by dates and categories

- As expected, this Apparel, Footwear & Accesories category, having a greater number of bids, could reflect a great variability. This is not always the case, but I can infer that bidders are willing to pay a lot of money because they know that this category is very profitable.
- This insight could be really usefull for BStock to know what category is more attractive for bids and a when is the right time to launch an auction!

In [722]:
import plotly.express as px


fig = px.box(df, x="bstock_category_name", y="bid_amount", title='Bid amount distribution per category and auction date')
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_xaxes(categoryorder='array', categoryarray=['Apparel, Footwear & Accessories', 'Consumer Electronics', 'Furniture'])
fig.show()

- In order to be impartial, I consider that it would be better to disaggregate these box plots to identify which where the dates with greatest bid amount variability.
- November 1st, December 13th and November November 4th were auction dates with highest bid amount variability!
- On those dates, only 'Apparel, Footwear & Accesories category were sold.

- On November 1st, the auction started with a bid amount of 125 USD and finished with a bid amount of 6222 USD
- On December 13th, the auction started with a bid amount of 444 USD and finished with a bid amount of 5925 USD
- ON November 4th, the auction started with a bid amount od 125 USD and finished with a bid amount of 4000 USD

In [723]:
import plotly.express as px


fig = px.box(df, x="auction_date", y="bid_amount", color="bstock_category_name", title='Bid amount distribution per category and auction date')
fig.update_traces(quartilemethod="exclusive")
fig.show()

# Monthly Analysis

- Now is the time to focus on business growth!
- Can we begin to ask ourselves how many bids per month we have?

In [724]:
# Truncate bid_date to month values
import datetime as dt
def get_month(x): return dt.datetime(x.year, x.month, 1)

df['bid_date_month'] = df['bid_date'].apply(get_month)
df

,auction_date,auction_id,bstock_category_name,bid_date,bid_id,bidder_id,bid_amount,bid_date_month
0,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:49,16819417,1,10.0,2022-09-01
1,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:50,16819418,1,100.0,2022-09-01
2,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:51,16819419,2,101.0,2022-09-01
3,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:52,16819420,1,200.0,2022-09-01
4,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:53,16819421,2,201.0,2022-09-01
...,...,...,...,...,...,...,...,...
822,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-13 23:58:16,18712359,5fbc4f53c8ce886dac7f4ba8,500.0,2022-12-01
823,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-14 02:55:27,18713339,59769fe3e3d4600d362c31c0,708.0,2022-12-01
824,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-14 10:12:08,18714207,5fbc4f53c8ce886dac7f4ba8,750.0,2022-12-01
825,2022-12-16,1721860,"Apparel, Footwear & Accessories",2022-12-15 16:13:16,18726173,601e76d8c8ce886da835b151,850.0,2022-12-01


- Well, with this limited dataset I can't conclude if there is any monthly seasonality
- But, I can conclude that November was the month with highest bids (324)!

In [725]:
# Total bids per month
bids_per_month = df.groupby(['bid_date_month'])
bids_per_month = bids_per_month.agg({'bid_id': ['count']})
bids_per_month.columns = ['bids']
bids_per_month

,bids
bid_date_month,
2022-09-01,251
2022-10-01,100
2022-11-01,324
2022-12-01,152


In [726]:
# Total bids per month
import plotly.express as px

fig = px.line(bids_per_month, x=bidders_per_month.index, y="bids", title='Bids per month')
fig.show()

- Also, we want to know how many active bidders do we have each month!
- On October 2022, we had the lowest active bidders (21)!
- On September 2022, we had the highest active bidders (52)!

In [727]:
# Total unique bidders per month
bidders_per_month = df.groupby(['bid_date_month'])
bidders_per_month = bidders_per_month.agg({'bidder_id': ['count'], 'bidder_id':pd.Series.nunique})
bidders_per_month.columns = ['bidders']
bidders_per_month

,bidders
bid_date_month,
2022-09-01,52
2022-10-01,21
2022-11-01,42
2022-12-01,22


In [728]:
# Total unique bidders per month
import plotly.express as px

fig = px.line(bidders_per_month, x=bidders_per_month.index, y="bidders", title='Bidders per month')
fig.show()

- Last but not least, to assess how fast our company grows we must know the first-time bidders for each month!
- On October 2022, we had the lowest first-time bidders (11)!
- On November 2022, we had 31 first-time bidders!
- On September 2022, we had the highest first-time bidders (52), was a great month!

In [729]:
# First-time bidders per month
grouping = df.groupby('bidder_id')['bid_date']
df['cohort_month'] = grouping.transform('min')
df['cohort_month'] = df['cohort_month'].apply(get_month)

first_time_bidder_per_month = df.groupby('cohort_month')
first_time_bidder_per_month = first_time_bidder_per_month.agg({'bidder_id': ['count'], 'bidder_id':pd.Series.nunique})
first_time_bidder_per_month.columns = ['first-time bidders']
first_time_bidder_per_month

,first-time bidders
cohort_month,
2022-09-01,52
2022-10-01,11
2022-11-01,31
2022-12-01,13


In [730]:
merge_bidders = pd.merge(bidders_per_month, first_time_bidder_per_month, how='inner', left_index=True, right_index=True)
merge_bidders

,bidders,first-time bidders
bid_date_month,,
2022-09-01,52,52
2022-10-01,21,11
2022-11-01,42,31
2022-12-01,22,13


# Bidders Analytics

- We have to know our bidders!
- Who is the bidder with highest historial bids?
- The bidder '62e329abff03c2000d83079d' is the most active customer in last months!
- We can give him/her/them a loyalty program with us!
- But, not so fast, let's find out if this bidder is profitable...

In [734]:
df.head()

,auction_date,auction_id,bstock_category_name,bid_date,bid_id,bidder_id,bid_amount,bid_date_month,cohort_month
0,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:49,16819417,1,10.0,2022-09-01,2022-09-01
1,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:50,16819418,1,100.0,2022-09-01,2022-09-01
2,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:51,16819419,2,101.0,2022-09-01,2022-09-01
3,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:52,16819420,1,200.0,2022-09-01,2022-09-01
4,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:53,16819421,2,201.0,2022-09-01,2022-09-01


In [732]:
# Historical bids per bidder ID
bidders_stats = df.groupby(['bidder_id'])
bidders_stats = bidders_stats.agg({'bid_id':['count']})
bidders_stats.columns = ['total bids']
bidders_stats = bidders_stats.sort_values(by='total bids', ascending=False)
bidders_stats


,total bids
bidder_id,
62e329abff03c2000d83079d,194
6359b3f4feb3d7000ca36001,57
60c94b4fc8ce886daa7749b4,38
620412efc8ce885c4ac7395f,28
5e9bf479c8ce886c85a237b9,23
...,...
5eac1301c8ce880620f02290,1
5e45a255c8ce886c859ecfee,1
5e44b8cbc8ce886c884331ec,1


In [735]:
# October bids per bidder ID

bidders_stats_oct = df[df['bid_date_month']=='2022-10-01'].groupby(['bidder_id'])
bidders_stats_oct = bidders_stats_oct.agg({'bid_id':['count']})
bidders_stats_oct.columns = ['october bids']
bidders_stats_oct = bidders_stats_oct.sort_values(by='october bids', ascending=False)
bidders_stats_oct

,october bids
bidder_id,
62e329abff03c2000d83079d,25
5e9bf479c8ce886c85a237b9,23
62a227deff03c2000b28e506,8
62de2eb5ff03c2000f137373,7
600ed3a0c8ce886dac844326,6
5f5bf9eec8ce880639fe9393,6
54c3586f421aa90e94a53497,5
54c356f5421aa90e92aab121,3
60f9ad9cc8ce886da7e41b40,2


- To keep things simple, I developed a query on MySQL to return the following report!
- That is an inner join between misc.abc_bids and misc.abc_orders with the winner or ua_id_buyer

In [683]:
bidders_orders = pd.read_csv('bidders-and-orders.csv')
bidders_orders.head()

,auction_date_transformed,auction_id,bstock_category_name,bid_date,bid_id,bidder_id,bid_amount_transformed,bstock_condition_name,winning_bid_usd,retail_price,profit
0,2022-09-10,1500000,"Apparel, Footwear & Accessories",2022-09-10 17:22:59.000000 UTC,16819428,1,1100,NaN,NaN,NaN,NaN
1,2022-09-13,1640483,"Apparel, Footwear & Accessories",2022-09-13 17:33:16.000000 UTC,17819741,62e329abff03c2000d83079d,3000,Brand New,2925.0,23195.0,20270.0
2,2022-09-13,1640484,"Apparel, Footwear & Accessories",2022-09-13 16:12:53.000000 UTC,17817684,62e329abff03c2000d83079d,207,Brand New,155.0,28526.0,28371.0
3,2022-09-13,1640485,Furniture,2022-09-13 15:48:56.000000 UTC,17817438,62e329abff03c2000d83079d,2521,Like New,2521.0,40122.0,37601.0
4,2022-09-13,1640486,Consumer Electronics,2022-09-13 16:59:30.000000 UTC,17818639,62e329abff03c2000d83079d,2667,Brand New,1900.0,46405.0,44505.0


- Indeed, this hidder (62e329abff03c2000d83079d) is the most active in our auctions (194 total bids) and is very profitable (526,881 USD)!
- Let's find out in which categories it is most profitable!

In [739]:
order_stats = bidders_orders.groupby(['bidder_id'])
order_stats = order_stats.agg({'profit':['sum']})
order_stats.columns = ['total profit']
order_stats = order_stats.sort_values(by='total profit', ascending=False)
order_stats

,total profit
bidder_id,
62e329abff03c2000d83079d,526881.0
60130b4ac8ce886da943b405,69639.0
6346cfecfeb3d7000dbb3495,44234.0
54c35b66421aa90e9061d305,43766.0
60c94b4fc8ce886daa7749b4,38706.0
635b4a04feb3d7000dbeacfc,35274.0
5872eb71421aa90ceab0bb2f,35025.0
591e47e3e3d4600d330ec1e8,32831.0
5e9bf479c8ce886c85a237b9,31425.0


- As expected, bidder ID '62e329abff03c2000d83079d' is very profitable in the 'Apparel, Footwear & Accesories category (396,680 USD)

In [745]:
order_stats_cat = bidders_orders[bidders_orders['bidder_id']=='62e329abff03c2000d83079d'].groupby(['bidder_id', 'bstock_category_name'])
order_stats_cat = order_stats_cat.agg({'profit':['sum']})
order_stats_cat.columns = ['total profit']
order_stats_cat = order_stats_cat.sort_values(by=['total profit'], ascending=False)
order_stats_cat

total profit
bidder_id                bstock_category_name                         
62e329abff03c2000d83079d Apparel, Footwear & Accessories      396680.0
                         Consumer Electronics                  92600.0
                         Furniture                             37601.0

- Last but not least, do you remember the following question?:
- "Is it possible that the sale of clothing, shoes and accessories generates a lot of profitability?"
- Let's find it out!

In [746]:
category_stats = bidders_orders.groupby(['bstock_category_name'])
category_stats = category_stats.agg({'profit':['sum']})
category_stats.columns = ['total profit']
category_stats = category_stats.sort_values(by='total profit', ascending=False)
category_stats

,total profit
bstock_category_name,
"Apparel, Footwear & Accessories",879628.0
Consumer Electronics,105772.0
Furniture,37601.0


- That's the reason why bidders pay a lot of money in the Apparel, Footwear & Accesories category because it is very profitable.
- In the last 4 months, our bidders registered a profit of 879,628 USD on that category!

In [747]:
fig = px.bar(category_stats, x=category_stats.index, y="total profit", title='Total profit per B-Stock Category')
fig.show()

# Highlights

1. Most of the auctions were held on Tuesdays and Fridays
2. September 13th was the one that registered the highest auctions amount (6) because all three categories were sold.
3. September 10th was the one that registered the lowest auctions amount (1) because only Apparel, Footwear & Accesories category was sold.
4. November 1st was the date with the most bids registered ever, even though there were only 2 auctions of 'Apparel, Footwear & Accesories' category.
5. 'Apparel, Footwear & Accesories' category has a total of 741 bids
6. 'Consumer Electrics' category has a total of 62 bids
7. 'Furtniture' category has a total of 24 bids
8. As expected, this Apparel, Footwear & Accesories category, having a greater number of bids, could reflect a great variability. This is not always the case, but I can infer that bidders are willing to pay a lot of money because they know that this category is very profitable.
9. On November 1st, the auction started with a bid amount of 125 USD and finished with a bid amount of 6222 USD
10. On December 13th, the auction started with a bid amount of 444 USD and finished with a bid amount of 5925 USD
11. ON November 4th, the auction started with a bid amount od 125 USD and finished with a bid amount of 4000 USD 
12. November 2022 was the month with highest bids (324)
13. October 2022 was the month with lowest bids (100)
14. On October 2022, we had the lowest active bidders (21)!
15. On September 2022, we had the highest active bidders (52)!
16. On October 2022, we had the lowest first-time bidders (11)!
17. On November 2022, we had 31 first-time bidders!
18. On September 2022, we had the highest first-time bidders (52), was a great month!
19. The bidder '62e329abff03c2000d83079d' is the most active customer (194 total bids) in last months!
20. The bidder '62e329abff03c2000d83079d' is the most profitable (526,881 USD)!
21. The bidder '62e329abff03c2000d83079d' is very profitable in the 'Apparel, Footwear & Accesories category (396,680 USD)
22. 'Apparel, Footwear & Accessories' category is the most profitable in the last month (879,628 USD total profit). That's the reason why bidders pay a lot of money in that category because it is very profitable.
